Last time, we used properties of the normal distribution to derive classical confidence intervals for things like averages (means). 

<img src=https://errorstatistics.files.wordpress.com/2014/10/normal-curve-2-3-and-5-sigma.jpg width=500>

We saw what we meant by the term "confidence" and how, because they invoke an imaginary set of repeated trials, they are part of the "frequentist" statistician's toolbox.

**Regression**

Today we'll start with an antique data set, Galton's heights. It's a simple 2-column data set that we can read in as usual. We will make some of the plots referred to in "lecture" and then fit Galton's regression model. 

We'll look at it's head and tail...

In [ ]:
galton <- read.csv("https://github.com/computationaljournalism/columbia2019/raw/master/data/galton.csv")

In [ ]:
dim(galton)
head(galton)

In [ ]:
tail(galton)

And let's look at Galton's original table. Notice that by default, R will order the rows and columns of the table in ascending order of its labels. That means the rows are ordered from short parents (at the top) to tall parents (at the bottom). Galton's table is upside down, making it more like a plot.

<img src=https://nielsen.sites.oasis.unc.edu/soci709/m1/m1021.jpg height=700>

In [ ]:
table(galton$midparent,galton$child)

Remember that Galton found the "marginals" of the table were normally distributed. That means, ignoring the relationships between rows and columns, histograms of midprents or childs heights look bell-shaped. What do you think?

(A relatively new, major graphics system (aside from special graphics that come along with new models, say) is `ggplot` -- well, `ggplot2.` It comes from Lee Wilkinson's book "Grammar of Graphics." In it, Wilkinson tried to come up with a system for describing any plot. It was meant to not only recreate existing graphical forms, but allow for the creation of new ones. 

`ggplot2` was created by, well it's getting old now but, Hadley Wickham. You can read [the formal documentation](http://docs.ggplot2.org/current/index.html). There is also [a great cheetsheat for ggplot2](https://www.rstudio.com/wp-content/uploads/2016/11/ggplot2-cheatsheet-2.1.pdf) and I recommend this [tutorial for ggplot2](http://zevross.com/blog/2014/08/04/beautiful-plotting-in-r-a-ggplot2-cheatsheet-3/).)

In [ ]:
library(dplyr)
library(ggplot2)
options(repr.plot.width=8, repr.plot.height=6)

ggplot() + geom_bar(aes(x=midparent),galton,color="white",fill="lightblue")

In [ ]:
ggplot() + geom_bar(aes(x=child),galton,color="white",fill="lightblue")

Recall he also observed that slicing along the table, selecting a single value of either midparent or child height will produce a bell curve. This is what we mean by the bivariate normal distribution.

In [ ]:
ggplot() + geom_bar(aes(x=child),filter(galton,midparent==67.5),color="white",fill="lightblue")

And now to fitting. R has a very expressive set of functions that give you access to a range of models. From simple linear regression to nonlinear fits to so-called generalized linear models and machine learning procedures. Many have the same basic syntax. Here we use the "formula language" to specify our model (children's heights as a function of midparent's heights) and have a look at the classic table.

In [ ]:
fit <- lm(child~midparent,data=galton)
summary(fit)

In [ ]:
print(coefficients(fit))

See how we can highlight just the parts of the output we want. Typically in a regression, we want to check things like the "residuals" to assess how well we've captured the main structure of the data.

In [ ]:
r <- residuals(fit)

ggplot() + geom_histogram(aes(x=r),bins=25,color="white",fill="lightblue")

We are going to use a data set associated with a [simple story from the NYT about the hardest places to live in the United States.](http://www.nytimes.com/2014/06/26/upshot/where-are-the-hardest-places-to-live-in-the-us.html?_r=0) It is county-level data. Quoting The UpShot, it consists of "six data points for each county in the United States: education (percentage of residents with at least a bachelor’s degree), median household income, unemployment rate, disability rate, life expectancy and obesity. We then averaged each county’s relative rank in these categories to create an overall ranking." Here's the data.

In [ ]:
hardest <- read.csv("http://graphics8.nytimes.com/newsgraphics/2014/06/16/worst-places/d92796c59951a5cccb3ad6411f599ad302a5c4bd/unemployment.tsv",sep="\t",na.strings=c("No Data","#N/A"))
head(hardest)

In [ ]:
sample_n(hardest,10)

In [ ]:
hardest <- na.omit(hardest)

Consult the NYT article for a discussion of the variables. How is the overall rank computed for each state? Why is it done this way? What do you think? Let's use some of our regression skills on this new data set.

In [ ]:
ggplot(hardest,aes(x =obesity,y=life))+geom_point()

In [ ]:
ggplot(hardest,aes(x =education,y=income))+geom_point()

Let's overlay the fitted line to the data. What do you see?

In [ ]:
ggplot(hardest,aes(x =obesity,y=life))+geom_point()+stat_smooth(method=lm,color="blue")

And let's check the residuals. How do things look?

In [ ]:
r <- residuals(fit)

ggplot() + geom_histogram(aes(x=r),bins=25,color="white",fill="lightblue")

Simple regression is not so simple. In Galton's case, everything lined up beautifully (pun intended). Often, we have to worry about what to do when the data do not follow a simple line. In multiple regression, we will also have to wonder which variables should be included. These questions set off entire reserch areas in statistics. 

To overcome the linear assumption, we can follow Galton's averaging approach. Here's (essentially) one example of how you can deal with it. 

In [ ]:
ggplot(hardest,aes(x =education,y=unemployment))+geom_point()+stat_smooth(method=loess,color="blue")

One handy plot from old school (not ggplot2) graphics is `pairs()` to see as many relationships at once as you can.

In [ ]:
pairs(select(hardest,education,unemployment,disability,life,obesity),pch=".")

**The bootstrap for regression**

Before we leave regression (or generalize it), I want to talk a little about the classical statistical line of story telling -- confidence intervals. Your introductory class was probably needlessly vague about this idea. We are going to talk about a procedure called the bootstrap. It swaps mathematical or analytical results for computational effort.

In [ ]:
fit <- lm(child~midparent,data=galton)
summary(fit)

The bootstrap is a simple way to derive confidence intervals. Again, "analyze as you randomized." In this case, we look to Galton's simple sampling to collect his heights data (there are other ways to run the simulation if this assumption isn't right). 

Here's the essential "resampling" step.

In [ ]:
n <- nrow(galton)
n

In [ ]:
boot <- sample_n(galton,n,replace=TRUE)
summarize(boot,coef=coefficients(lm(child~midparent))[2])

And here's our loop, repeating our analysis 5,000 times. We can then look at the bootstrap sample, the estimate of the sampling distribution, and see how it compares to our table.

In [ ]:
# we'll store the summary statistics in boots

boots <- NULL

# repeat this 5000 times, building up boots as we go

for(i in seq(5000)){
    
    boot <- sample_n(galton,n,replace=TRUE)
    
    boots <- bind_rows(boots,
                       summarize(boot,coef=coefficients(lm(child~midparent))[2]))
}

In [ ]:
ggplot() + geom_histogram(aes(x=coef),boots,bins=25,color="white",fill="lightblue")

In [ ]:
sd(pull(boots,coef))

In [ ]:
print(quantile(pull(boots,coef),c(0.025,0.975)))

In [ ]:
fit <- lm(child~midparent,data=galton)
summary(fit)

Notice that the bootstrap distribution looks normal and that its standard deviation matches that computed under the classical linear model. That's the great thing about the bootstrap -- when there is "math" to derive (with large enough sample sizes) confidence intervals, the bootstrap intervals will be essentially the same. The advantage you get with the bootstrap is that you can apply the bootstrap more broadly.

**Learning**

We are going to consider a general purpose learning machine. It was developed in large part by Leo Breiman -- he's written about the "two cultures" in modeling. A **decision tree** is statistical model that is at one time a useful visualization as well as a prediction engine.  

The data comes from the election in 2008, and specifically during the primaries. It was developed by Amanda Cox, an outstanding visual journalist at The New York Times.

![tree](https://static01.nyt.com/images/2008/04/16/us/0416-nat-subOBAMA.jpg)

Have a look. The tree is like a game of 20 questions. If you want to know how a county will vote, answer a few questions. What is the racial makeup? What about the HS graduation rate? Where in the country is the county? 

The questions are in the form of a tree -- as we have seen before, computer scientists grow their trees upside down. As you answer questions, you eventually end up at a leaf and a decision is made. Note that it's not perfect -- there is some uncertainty in the decision. 

To show that this isn't a one-off, ProPublica had a lovely project called the [Message Machine](https://projects.propublica.org/emails/) where they looked email messages sent out by [political campaigns](https://www.propublica.org/special/message-machine-you-probably-dont-know-janet) and [reverse engineered the logic that generated them.](https://www.propublica.org/article/message-machine-starts-providing-answers)


Let's load up the data and see how such a thing comes into existence.

In [ ]:
library(rpart)
library(dplyr)
library(ggplot2)

In [ ]:
primary <- read.csv("https://raw.githubusercontent.com/computationaljournalism/columbia2019/master/data/primary.csv",as.is=TRUE)

head(primary)
dim(primary)

In [ ]:
names(primary)

At the point this graphic was published Montana, Oregon, South Dakota, Indiana, Kentucky, West Virginia, Pennsylvania, and South Carolina had yet to have their primaries. In all, there should be 2,261 counties for analysis. Here's where NY state had landed.

In [ ]:
select(filter(primary,state_postal=="NY"),winner,county_name)

And here is how the various counties across the country voted.

In [ ]:
table(select(primary,winner))

Now, given all these potential predictor variables, which ones "explain" county-level voting patterns? Suppose, for example, we start simply, and consider whether or not a majority of the county voted for Bush in 2004. 

In [ ]:
table(select(primary,winner,pres04winner))

In [ ]:
290/(290+163)

In [ ]:
1047/(1047+740)

Therefore, Obama won about 64% of those counties not voting for Bush in 04, while Clinton won about 59% of those counties that did vote for Bush in 04. Now, consider the **simple prediction rule**: If a county voted for Bush in 04, we’ll say that they will vote for Clinton in the primary, while if a county was mostly in favor of Kerry in 04, we’ll assign the win to Obama. 

Of course this rule isn’t perfect; by applying it, we would make 163 + 740 =
903 mistakes (out of 2,261 counties, or about 40% error); we refer to these mistakes as having been "misclassified" by our simple rule. So the question becomes, can we do any better? What do you think?

In [ ]:
(163 + 740)

In [ ]:
(163 + 740)/2261

There might be better indicators of Obama’s success besides the vote in 2004 -- but how do we find them? The decision tree encodes a large search across the complete data set for predictors that are informative. 

Consider the top of the tree, "the root". Decision trees work by repeatedly splitting the data into two parts; the root or first "split" is the single division of the data into two pieces that produces the lowest misclassification error (well, it's a little more complicated, but this will do).

To investigate this a little further, let’s consider the predictor that represents the
percentage of a county that is African American; we now choose a breakpoint that divides the data into two pieces (those counties with a greater percentage of African Americans and those with a smaller percentage). We then form a table (as we did for counties that went for Bush or Kerry in 2004) and count the misclassification rate.

Suppose we take 10% as the cutoff. We get the following table. How many errors do we make? What is the misclassification rate?

In [ ]:
table(pull(primary,winner),pull(primary,black06pct) > 0.1)

In [ ]:
# Error rate?


The 20% figure at the root of the tree was obtained by finding the magic point that minimizes the misclassification errors. In fact, the search was conducted over all the variables in the data set and all the possible splits; and this choice produced the smallest error.

Once this node has been chosen, we work our way down the tree, conducting the same search but on the specified subsets of the data, at each step attempting to minimize our errors. 

For quantitative variables, we have a choice of split points, and so the algorithm goes through a search. Here we vary a cutoff for the percentage of African Americans living in a county from essentially 0 to 0.5. For each choice of cutoff, we look at the associated error. 

What do you see?

In [ ]:
fractions <- (1:500)/1000

n <- length(fractions)
error <- NULL

for(i in 1:n){
    
    # right branch
    tmp <- select(filter(primary,black06pct > fractions[i]),winner)
    error.right <- min(table(tmp))
    
    # left branch
    tmp <- select(filter(primary,black06pct <= fractions[i]),winner)
    error.left <- min(table(tmp))
    
    error <- bind_rows(error,c(frac=fractions[i],err=error.left+error.right))
}

In [ ]:
options(repr.plot.width=8, repr.plot.height=5)

ggplot()+geom_line(aes(x=frac,y=err),error,xlab="percentage black",ylab="misclassification error")

In [ ]:
range(pull(primary,Bush04),na.rm=TRUE)

In [ ]:
fractions <- (100:900)/1000

n <- length(fractions)
error <- NULL

for(i in 1:n){
    
    # right branch
    tmp <- select(filter(primary,Bush04 > fractions[i]),winner)
    error.right <- min(table(tmp))
    
    # left branch
    tmp <- select(filter(primary,Bush04 <= fractions[i]),winner)
    error.left <- min(table(tmp))
    
    error <- bind_rows(error,c(frac=fractions[i],err=error.left+error.right))
}

In [ ]:
ggplot()+geom_line(aes(x=frac,y=err),error,xlab="Bush margin",ylab="misclassification error")

This is essentially what the decision tree process does. It's a greedy algorithm, meaning at each step, it is looking for the best way to divide the data into two pieces. There are things we can do to speed it up computationally, and there are issues with really trying to minimize misclassification rates, but this is essentially the approach.

Stop and think what this simple process has produced for us; we have a very intuitive structure (something akin to the game 20 questions) that makes evident "important" variables that help "explain" voting patterns. This kind of tool lives somewhere between data analysis and modeling; it is technically a model all by itself (making predictions) but is often used as a way to identify important predictors for another stage of model.

This decision tree is part of a large class of methods called CART for Classification and Regression Trees and was developed in the 1980s as part of a move to deal with bigger and meaner data sets.

Here is the implementation in R.

In [ ]:
fit <- rpart(winner~region+pres04margin+black06pct+hisp06pct+white06pct+
            growth+pct_less_30k+pct_more_100k+pct_hs_grad+pct_homeowner+POP05_SQMI,data=primary)

In [ ]:
par(xpd=TRUE)

plot(fit)
text(fit,use.n=TRUE,cex=0.8)

Let’s think a bit more about the tree-growing process; with each split, we cut
our data at the node into two pieces so that the “sample size” at each of the child nodes is lower than its parents. We then represent the data in the leaves with a simple model; for our 0/1 data (Obama or Clinton), we classify leaves according to majority vote.

In principle we can grow trees until there’s a single entry in each node -- What
might be the problem with that? How do we decide to stop splitting? When we
run out of data? 

To grow a bigger tree, we can reduce "cp", the complexity parameter. Any split that does not decrease the overall lack of fit by a factor of cp is not attempted by our algorithm, so making it smaller will produce a larger tree.

In [ ]:
fit <- rpart(winner~region+pres04margin+black06pct+hisp06pct+white06pct+
            growth+pct_less_30k+pct_more_100k+pct_hs_grad+pct_homeowner+POP05_SQMI,data=primary,cp=0.00001)

In [ ]:
plot(fit)

In the R displays above, the heights of the branches correspond to the error in represented by the model. So, the at the root, we are dealing with all the counties -- 1210 went for Clinton
and 1031 for Obama -- therefore since Clinton won more counties, we would
predict all future counties for Clinton, making 1030 mistakes.

The first split on the percentage of the county that is
African American brought us down to 700 errors, a big drop; the next division
based on education is another big drop, about half the size. As we continue to refine the tree, however, the improvements diminish.

In the mid-1980s a fair bit of theoretical and methodological work was devoted
to understanding the behavior of this kind of algorithm; we are using it as a bit
of data analysis (how does a “response” relate to the potential “explanatory”
variables?) but it can also be used as a tool for making predictions.

The R command rpart (for recursive partitioning) employs a “pruning”
algorithm that divides the original data into several parts; iteratively leaving one
part out, building a tree and evaluating it on the part that was left out -- this is
called cross validation.

In [ ]:
plotcp(fit)

In [ ]:
printcp(fit)

In [ ]:
plot(prune(fit,0.01))

A lovely graphical introduciton to decision trees is [given here.](http://www.r2d3.us/visual-intro-to-machine-learning-part-1/)

**More recent data**

Here is a data set collected by Loren Collingwood, a political scientist. It accumulates many of the same county-level variables but is looking at the recent national election instead. Let's try out our Rpart skills!

In [ ]:
election = read.csv("http://www.collingwoodresearch.com/uploads/8/3/6/0/8360930/county_data.csv",sep="\t")
head(election)

Make a column indicating the winner for each county.

In [ ]:
election = mutate(election,winner = ifelse(pct_clinton < pct_trump,"Trump","Clinton"))
head(election)

In [ ]:
fit = rpart(winner~per_capita_income+pobama12cnty+percent_white,data=election)

In [ ]:
par(xpd=TRUE)

plot(fit)
text(fit,use.n=TRUE,cex=0.8)

Collingwood didn't have a dictionary for his data, but he was able to answer a few questions. Here are the columns I couldn't read easily and his explanations.

totpop1014cnty what does 1014 refer to? 1014 IS SAMPLED BETWEEN YEARS 2010-14 FOR ACES
<br>
ppiwht1014cnty   ppi? --I'M PRETTY SURE THIS IS PERCENT WHITE, THN PERCENT BLACK, ETC.
<br>
ppiblk1014cnty
<br>
ppihisp1014cnty
<br>
pimm1014cnty        mm? PERCENT IMMIGRANT
<br>
ppi0014cnty         ppi again and what is 0014? THE 0014 WOULD NOW BE A CHANGE MEASURE -- PERCENT CHANGE FROM 2000 TO 2014.
<br>
psocsec0014cnty
<br>
ownthirty0014cnty   ownthirty?
<br>
ownfifty0014cnty
<br>
pmanufact1014cnty   ? employment in manufacturing? CORRECT -- THIS WAS PREDICTOR OF TRUMP VOTE
<br>
pct65over00cnty     this is in 2000? YES, FOR OLD PEOPLE
<br>
medoohvalue00cnty    ? MEDIAN HOME VALUE 2000
<br>
pctcomlong00cnty    comlong? I THINK THIS IS COMMUTE OVER 30 MINS -- BUT I DIDN'T USE THIS SO NOT 100% ON THAT.
<br>
pctdiffh95_00cnty   ? NOT SURE
<br>
pctfb00cnty         ? FOREIGN BORN
<br>
pcpi00cnty           pcpi? NOT SURE
<br>
Rpt.                ? NOT SURE
<br>
state.y STATE NAME/CODE. PROBABLY A state.x AS WELL? FUNCTION OF MERGING IN R
<br>
pct_change_R         ?
<br>
manu_loss            ? IS THIS A DUMMY? IF SO, DID THE COUNTY RECEIVE MANUFACTURING LOSS IF YES = 1, ELSE = 0
<br>
rural                ? RURAL COUNTY BASED OFF OFFICIAL CENSUS DESIGNATION (DUMMY)
<br>
rural_150            ? SIMILAR MEASURE, THIS IS THE ONE I THINK IS BASED OF 100 PER SQ/MILE. BUT WOULD HAVE TO CHECK